<a href="https://colab.research.google.com/github/raketic-ognjen/introtoportfolioopt2/blob/main/Introtoportfolioopt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='tomato' style="font-size:40px"><center><b>Introduction to Portfolio Optimization</b></center></font><font color='DarkOrange' style="font-size:30px"><center><b>- Part 2 -</b></center></font>

## <font color='orange' style="font-size:25px"><b>Installing and updating relevant packages</b></font>

As usual, I need to update several packages which older version already exist in Google Colab:

In [2]:
%pip install plotly

Before I start today's coding practice, it is necessary to import following packages:
* <font color='mediumseagreen'><b>Pandas</b></font>
* <font color='mediumseagreen'><b>NumPy</b></font>
* <font color='mediumseagreen'><b>Plotly</b></font>
* <font color='mediumseagreen'><b>SciPy - Stats</b></font>
* <font color='mediumseagreen'><b>SciPy - Optimize</b></font>
* Our small package for investment lectures - <font color='DarkTurquoise'><b>quant formulas</b></font>. Before we import it, we need to upload it, of course:

In [4]:
from google.colab import files
uploaded0=files.upload()

Saving kvantitativne_formule.py to kvantitativne_formule.py


In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import scipy.stats as st
import scipy.optimize as sco
import kvantitativne_formule as qt

Also, before I start, I need to prepare data in the same way as I did in all my repositories. Thus, let me first import file "optimization.xlsx":

In [6]:
from google.colab import files
uploaded1= files.upload()

Saving optimization.xlsx to optimization.xlsx


In [11]:
prices=pd.read_excel('optimization.xlsx',sheet_name='Sheet1',index_col="DATE")
prices

,AMZN,MSFT,IBM,AAPL,^GSPC,rf
DATE,,,,,,
2013-1-2,257.309998,27.620001,196.350006,78.432899,1462.420044,0.000007
2013-1-3,258.480011,27.250000,195.270004,77.442299,1459.369995,0.000007
2013-1-4,259.149994,26.740000,193.990005,75.285698,1466.469971,0.000007
2013-1-7,268.459015,26.690001,193.139999,74.842903,1461.890015,0.000006
2013-1-8,266.380005,26.549999,192.869995,75.044296,1457.150024,0.000006
...,...,...,...,...,...,...
2014-12-24,303.029999,48.139999,161.820007,112.010002,2081.879883,0.000005
2014-12-26,309.089996,47.880001,162.339996,113.989998,2088.770020,0.000005
2014-12-29,312.040009,47.450001,160.509995,113.910004,2090.570068,0.000005


In [13]:
modellingP=prices[:300] #the first 300 obervations
testingP=prices[300:] #rest

In [14]:
retM=modellingP.iloc[:,:4].pct_change()
retM.dropna(inplace=True)
retM.head()

,AMZN,MSFT,IBM,AAPL
DATE,,,,
2013-1-3,0.004547,-0.013396,-0.005500,-0.012630
2013-1-4,0.002592,-0.018716,-0.006555,-0.027848
2013-1-7,0.035921,-0.001870,-0.004382,-0.005882
2013-1-8,-0.007744,-0.005245,-0.001398,0.002691
2013-1-9,-0.000113,0.005650,-0.002852,-0.015629


## <font color='Orange' style="font-size:25px"><b>Efficient Frontier</b></font>

We start, for simplicity, with an example of 2 stocks. We assume that no short selling is allowed, i.e. that weights are between 0 and 1. **You can read Bodie, Kane, Markus (pp 197-205) for some additional insights into the problem with two assets.**

### <font color='MediumVioletRed' style="font-size:20px"><b>Initial discussion - portfolio possibilities curve</b></font>

Assume that you have only two stocks in your portfolio. Variances of their returns are given as: $\mathbb{V}\left(r_{1}\right)=\sigma_1^2$ and $\mathbb{V}\left(r_{2}\right)=\sigma_2^2$, while covariance of their returns is $\sigma_{1,2}$. In this case, portfolio's variance would be (recall that $\alpha_1 + \alpha_2 = 1$):

$$\mathbb{V}(r_{p})= \alpha_1^2\sigma_1^2+\alpha_2^2\sigma_2^2+2\,\alpha_1\,\alpha_2\,\sigma_{1,2}$$

Note that covariance (just like the expected returns or standard deviations) depend on the time units that we use. If we want to have a measure of linear statistical dependence which does not depend on units we use correlation coefficients. Correlations are always between -1 (perfect negative correlation) to +1 (perfect positive correlation).

 Correlation coefficient $\rho$ is defined as:

$$\rho=\frac{\sigma_{1,2}}{\sigma_1\,\sigma_2}\,\,\,\,\,\,\,\,\,\, \Longleftrightarrow\,\,\,\,\,\,\,\,\,\, \sigma_{1,2}=\,\rho\, \sigma_1\,\sigma_2$$